# About

__Notes:__

- We will create experiments with different configurations
- Then we will compare metrics for different experiments using _DVC_

# Experiment 1 - Tune LogisticRegression

## Create branch for experiment

```bash
git checkout -b exp1-tune-logreg
git branch
```

## Update config/pipeline_config.yml file

add options for **C** hyperparamter in section `train:estimators:logreg:param_grid`:


```yaml
...
        param_grid:
              C: [0.1,1.0,10]
...
```

as result you should have LogisticRegression config:

```yaml
...
train:
  cv: 3
  estimator_name: logreg

  estimators:

    logreg: # sklearn.linear_model.LogisticRegression
      param_grid: # params of GridSearchCV constructor
        C: [0.1,1.0,10]
        max_iter: [100]
        solver: ['lbfgs']
        multi_class: ['multinomial']
...

```


## Run experiment and save results 

Reproduce pipeline with new params

```bash
dvc repro 
```

## Commit experiment results

```bash
git add .
git commit -m "Experiment 1 with LogisticRegression hyperparameters"
git tag -a "exp1_tune_logreg" -m "Experiment 1 with LogisticRegression hyperparameters"
dvc push
git push origin exp1-tune-logreg
```

## Show metrics 

```bash
dvc metrics show
```

## Checkout back to `experiments` and pull last changes:

```bash
git checkout experiments
git pull origin experiments

```

# Experiment 2 - Use SVM

## Create branch for experiment

```bash
git checkout -b exp2-svm
git branch
```

## Add SVC config to config/pipeline_config.yml file

in section `train:estimators:logreg:param_grid`:


```yaml
...
        param_grid:
              C: [0.1,1.0,10]
...
```

as result you should have SVC config:

```yaml
...
train:
  cv: 3
  estimator_name: svm
  estimators:
        
    svm: # sklearn.svm.SVC
      param_grid:
        C: [0.1,1.0,10]
        kernel: ["rbf", "linear"]
        gamma: ["scale"]
        degree: [3, 5]
...
```


## Run experiment and save results 

Reproduce pipeline with new params

```bash
dvc repro
```

## Commit experiment results

```bash
git add .
git commit -m "Experiment 2 with SVM estimator"
git tag -a "exp2_svm" -m "Experiment 2 with SVM estimator"
dvc push
git push origin exp2-svm
```

## Show metrics 

```bash
dvc metrics show
```

## Do Merge on GitLab

# Compare experiments

__List experiments__

```bash
git tag --list
```

__View and compare metrics__

1) Last experiment metrics:

```bash
dvc metrics show
```

2) View and compare metrics for all experiments:

```bash
dvc metrics show -a
```

3) Сontrol metrics view 

```bash
dvc metrics show --show-json -a
```

4) View and compare metrics for all tags:

```bash
dvc metrics show -T
```
